**Imports and Setup**

In [1]:
pf_id = 2
# n_objectives = 3
n_scenarios = 5

In [2]:
import functools
import pandas as pd
from ema_workbench import (Model, RealParameter, ScalarOutcome,
                           MultiprocessingEvaluator, ema_logging,
                           Constant, SequentialEvaluator)
from ema_workbench import Samplers
from ema_workbench.em_framework.optimization import (HyperVolume,
                                                     EpsilonProgress)

import numpy as np

In [3]:
from problem_formulation import get_model_for_problem_formulation
from ema_workbench.util import ema_logging
from ema_workbench import Policy

ema_logging.log_to_stderr(ema_logging.INFO)

model, planning_steps = get_model_for_problem_formulation(pf_id) #changes for each file

def get_do_nothing_policies(l_bol = 0, policy_id = 0):
    return [Policy('policy %s'%policy_id, **{l.name:l_bol for l in model.levers})]

policies = get_do_nothing_policies()
policies

[Policy({'0_RfR 0': 0, '0_RfR 1': 0, '0_RfR 2': 0, '1_RfR 0': 0, '1_RfR 1': 0, '1_RfR 2': 0, '2_RfR 0': 0, '2_RfR 1': 0, '2_RfR 2': 0, '3_RfR 0': 0, '3_RfR 1': 0, '3_RfR 2': 0, '4_RfR 0': 0, '4_RfR 1': 0, '4_RfR 2': 0, 'EWS_DaysToThreat': 0, 'A.1_DikeIncrease 0': 0, 'A.1_DikeIncrease 1': 0, 'A.1_DikeIncrease 2': 0, 'A.2_DikeIncrease 0': 0, 'A.2_DikeIncrease 1': 0, 'A.2_DikeIncrease 2': 0, 'A.3_DikeIncrease 0': 0, 'A.3_DikeIncrease 1': 0, 'A.3_DikeIncrease 2': 0, 'A.4_DikeIncrease 0': 0, 'A.4_DikeIncrease 1': 0, 'A.4_DikeIncrease 2': 0, 'A.5_DikeIncrease 0': 0, 'A.5_DikeIncrease 1': 0, 'A.5_DikeIncrease 2': 0})]

In [4]:

# with SequentialEvaluator(model) as evaluator:
#     results = evaluator.perform_experiments(scenarios=n_scenarios,               #500
#                                             policies=4,
#                                             uncertainty_sampling= Samplers.MC) #monty (python) carlo

In [5]:

with SequentialEvaluator(model) as evaluator:
    results, convergence = evaluator.optimize(nfe=100, searchover='uncertainties', policies=policies, epsilons=[0.001, 0.001, 0.001, 0.001, 0.001])


  0%|                                                  | 0/100 [00:00<?, ?it/s][MainProcess/ERROR] 'DikeIncrease 0'
Traceback (most recent call last):
  File "C:\Users\Michiel\anaconda3\envs\gds\lib\site-packages\ema_workbench\em_framework\experiment_runner.py", line 92, in run_experiment
    model.run_model(scenario, policy)
  File "C:\Users\Michiel\anaconda3\envs\gds\lib\site-packages\ema_workbench\util\ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
  File "C:\Users\Michiel\anaconda3\envs\gds\lib\site-packages\ema_workbench\em_framework\model.py", line 350, in run_model
    outputs = self.run_experiment(experiment)
  File "C:\Users\Michiel\anaconda3\envs\gds\lib\site-packages\ema_workbench\util\ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
  File "C:\Users\Michiel\anaconda3\envs\gds\lib\site-packages\ema_workbench\em_framework\model.py", line 403, in run_experiment
    model_output = self.function(**experiment)
  File "C:\Users\Michiel\D

EMAError: exception in run_model
Caused by: KeyError: 'DikeIncrease 0'

***Save Outputs***

**Narrow Down Using epsilon progress and hypervolume**

In [ ]:
model.outcomes = [ScalarOutcome('Expected Annual Damage', kind=ScalarOutcome.MINIMIZE,
                                     expected_range=(0,5)),
                       ScalarOutcome('Dike Investment Costs', kind=ScalarOutcome.MINIMIZE,
                                     expected_range=(0,2)),
                       ScalarOutcome('RfR Investment Costs', kind=ScalarOutcome.MINIMIZE,
                                    expected_range=(0,1)),
                       ScalarOutcome('Evacuation Costs', kind=ScalarOutcome.MINIMIZE,
                                     expected_range=(0,1)),
                    ScalarOutcome('Expected Number of Deaths', kind=ScalarOutcome.MINIMIZE,
                                     expected_range=(0,1))]


In [ ]:
convergence = [HyperVolume(minimum=[0,0,0,0,0], maximum=[1, 1, 1, 1, 1]),
               EpsilonProgress()]

**Robustness**

**Robustness Function**

In [ ]:
import functools
import pandas as pd

def robustness(direction, threshold, data):
    if direction == SMALLER:
        return np.sum(data<=threshold)/data.shape[0]
    else:
        return np.sum(data>=threshold)/data.shape[0]

SMALLER = 'SMALLER'

Expected_Annual_Damage = functools.partial(robustness, SMALLER, 1000) #THOSE NUMBERS NEED TO BE SPECIFIED AGAINS
Dike_Investment_Costs = functools.partial(robustness, SMALLER, 1000)
RfR_Investment_Costs = functools.partial(robustness, SMALLER, 1000)
Evacuation_Costs = functools.partial(robustness, SMALLER, 1000)
Expected_Number_of_Deaths = functools.partial(robustness, SMALLER, 0.000001) #not ok

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
fig, axes = plt.subplots(ncols=2, nrows=3, figsize=(10,10),
                        sharex=True)
axes = [axes[0,0],axes[0,1],axes[1,0],axes[1,1]]                             #axes[1,1]

robustness_funcs = {"Expected Annual Damage": Expected_Annual_Damage,
                    "Dike Investment Costs": Dike_Investment_Costs,
                    "RfR Investment Costs": RfR_Investment_Costs,
                    "Evacuation Costs": Evacuation_Costs,
                    "Expected Number of Deaths": Expected_Number_of_Deaths}

experiments, outcomes = results

for ax, (outcome, value) in zip(axes, outcomes.items()):
    for policy in np.unique(experiments['policy']):
        logical = experiments['policy'] == policy
        data = value[logical]
        
        robustness = []
      
        for i in range(1, data.shape[0]):
            robustness.append(robustness_funcs[outcome](data[0:i]))
        ax.plot(robustness, label=policy)

    ax.set_xlabel("# experiments")
    ax.set_ylabel(outcome)